In [1]:
import os.path as op
import os
import numpy as np
import nibabel as nb
from ipyparallel import Client

In [2]:
engines = Client()

In [3]:
home_dir =  '/home/dnl/habitization/'
data_dir = home_dir + '/data/'
subs = home_dir + 'subjects.txt'
subs = list(np.loadtxt(subs,str))

sessions = ['a','b']
runs = map(str,range(1,7))

ignore_subs = ['HAB17','HAB18']
subs = [x for x in subs if x not in ignore_subs]

In [4]:
for sub in subs:
    for sesh in sessions:
        for run in runs:

            template = op.join(home_dir,'templates','glm.fsf')
            new_f = op.join(home_dir,'templates','glms','_'.join([sub,sesh,run]) +'.fsf')

            with open(template,'r') as file:
                filedata = file.read()

            filedata = filedata.replace('$SUB',sub)
            filedata = filedata.replace('$RUN',run)
            filedata = filedata.replace('$SESSION',sesh)
            
            #next get number of time points
            func = op.join(home_dir,'data',sub,'func','run_' + sesh + run + '.nii.gz')
            func = nb.load(func)
            ntpts = func.shape[-1]
            filedata = filedata.replace('$NUM_TIMEPOINTS',str(ntpts))

            
            with open(new_f,'w') as file:
                file.write(filedata)


In [5]:
def feat(in_tuple):
    sub,sesh,run = in_tuple
    fsf = op.join(home_dir,'templates','glms','_'.join([sub,sesh,run]) +'.fsf')
    
    cmd = ['feat',fsf]
    cmd = ' '.join(cmd)
    os.system(cmd)
    return in_tuple

In [6]:
in_tuples = []
for sub in subs:
    for sesh in sessions:
        for run in runs:
            in_tuples.append((sub,sesh,run))

In [7]:
sub_engines = engines[0:12] #less if you want to use fewer
sub_engines.push(dict(home_dir = home_dir))
sub_engines.execute('import os.path as op')
with sub_engines.sync_imports():
    import os
    
output = sub_engines.map_sync(feat,in_tuples)

importing os on engine(s)


# Now run fixed effects analyis across runs within each session